In [1]:
import pandas as pd

In [ ]:
drug_indication = pd.read_csv(r"...path_to\RDsqr-KG\Preprocessed_datasets\drug_disease_ttd.csv")
drug_map = pd.read_csv(r"...path_to\RDsqr-KG\Datasets\Vocabulary_links\drug links.csv")
drug_ttd_map = pd.read_csv(r"...path_to\RDsqr-KG\Preprocessed_datasets\drug_crosslinks_ttd.csv")
disease_map = pd.read_csv(r"...path_to\RDsqr-KG\Preprocessed_datasets\disease_references.csv")

In [3]:
drug_indication.head(1)

,TTD_ID,drug_name,disease_name,ICD_11,status
0,DZB84T,Maralixibat,Pruritus,EC90,Approved


In [4]:
merge1 = pd.merge(drug_ttd_map,drug_map,left_on='cas_id',right_on='CAS Number',how='left')
merge2 = pd.merge(merge1, drug_indication, left_on ='TTD_ID', right_on='TTD_ID', how='left')

In [5]:
drug_map.head(1)

,DrugBank ID,Name,CAS Number,Drug Type,KEGG Compound ID,KEGG Drug ID,PubChem Compound ID,PubChem Substance ID,ChEBI ID,PharmGKB ID,...,GenBank ID,DPD ID,RxList Link,Pdrhealth Link,Wikipedia ID,Drugs.com Link,NDC ID,ChemSpider ID,BindingDB ID,TTD ID
0,DB00001,Lepirudin,138068-37-8,BiotechDrug,NaN,D06880,NaN,46507011.0,NaN,PA450195,...,NaN,11916,http://www.rxlist.com/cgi/generic/lepirudin.htm,NaN,Lepirudin,http://www.drugs.com/cdi/lepirudin.html,NaN,NaN,NaN,DAP000541


In [6]:
drug_disease = merge2[['DrugBank ID', 'Name', 'ICD_11', 'disease_name', 'status']]

In [7]:
drug_disease.head(1)

,DrugBank ID,Name,ICD_11,disease_name,status
0,DB14570,Hydroxyprogesterone,2A00-2F9Z,Solid tumour/cancer,Approved


Disease Mapping

In [8]:
disease_map.head(1)

,OrphaCode,DisorderName,ICD-11,ICD-10,OMIM,UMLS,MeSH,MedDRA,GARD
0,166024,"Multiple epiphyseal dysplasia, Al-Gazali type",LD24.61,Q77.3,607131.0,C4304500,NaN,NaN,NaN


In [9]:
disease_map['DisorderName'] = disease_map['DisorderName'].str.replace(r'^NON RARE IN EUROPE:\s*', '', regex=True).str.strip()

In [10]:
df_disease_map = pd.merge(
    drug_disease, 
    disease_map, 
    left_on=['disease_name'], 
    right_on=['DisorderName'],  
    how='left'
)

In [11]:
df_orpha = df_disease_map.rename(columns={'DrugBank ID': 'head_id', 'Name': 'head_name', 'OrphaCode': 'tail_id','DisorderName':'tail_name'})

In [12]:
df_orpha['head_type'] = 'drug'
df_orpha['head_ref'] = 'DrugBank'
df_orpha['tail_type'] = 'disease'
df_orpha['tail_ref'] = 'Orphanet'
df_orpha['predicate'] = 'ddii'

In [13]:
df_orpha = df_orpha[['head_id', 'head_name', 'head_type','head_ref','tail_id', 'tail_name','tail_type','tail_ref','predicate', 'status']]

In [14]:
df_orpha.shape

(14287, 10)

In [15]:
df_orpha.isna().sum()

head_id       7403
head_name     7403
head_type        0
head_ref         0
tail_id      12839
tail_name    12839
tail_type        0
tail_ref         0
predicate        0
status        4810
dtype: int64

In [16]:
df_orpha.dropna(inplace=True)

In [17]:
df_orpha.drop_duplicates(inplace=True)

In [18]:
df_orpha.isna().sum()

head_id      0
head_name    0
head_type    0
head_ref     0
tail_id      0
tail_name    0
tail_type    0
tail_ref     0
predicate    0
status       0
dtype: int64

In [19]:
df_orpha.shape

(1030, 10)

In [ ]:
df_orpha.to_csv(r'...path_to\RDsqr-KG\KG_datasets\orpha.csv', index=False)

In [21]:
unmapped_rows = df_disease_map[df_disease_map['DisorderName'].isna()]

In [22]:
unmapped_rows.head()

,DrugBank ID,Name,ICD_11,disease_name,status,OrphaCode,DisorderName,ICD-11,ICD-10,OMIM,UMLS,MeSH,MedDRA,GARD
0,DB14570,Hydroxyprogesterone,2A00-2F9Z,Solid tumour/cancer,Approved,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,DB00898,Ethanol,GC00,Cystitis,Approved,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,DB00898,Ethanol,MG30,Chronic pain,Approved,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,DB00632,Docosanol,1F00,Herpes simplex virus infection,Approved,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,DB01608,Periciazine,6E8Z,Psychiatric disorder,Approved,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
unmapped_rows.shape

(12839, 14)

In [24]:
unmapped_rows = unmapped_rows.rename(columns={'DrugBank ID': 'head_id', 'Name': 'head_name','ICD_11': 'tail_id', 'disease_name': 'tail_name'})

In [25]:
unmapped_rows['head_type'] = 'drug'
unmapped_rows['head_ref'] = 'DrugBank'
unmapped_rows['tail_type'] = 'disease'
unmapped_rows['tail_ref'] = 'ICD-11'
unmapped_rows['predicate'] = 'ddii'

In [26]:
unmapped_rows = unmapped_rows[['head_id', 'head_name', 'head_type','head_ref','tail_id', 'tail_name','tail_type','tail_ref','predicate', 'status']]

In [27]:
unmapped_rows.isna().sum()

head_id      6999
head_name    6999
head_type       0
head_ref        0
tail_id      4812
tail_name    4812
tail_type       0
tail_ref        0
predicate       0
status       4810
dtype: int64

In [28]:
unmapped_rows.dropna(inplace=True)

In [29]:
unmapped_rows.drop_duplicates(inplace=True)

In [30]:
unmapped_rows.shape

(4954, 10)

In [31]:
df_concat = pd.concat([df_orpha, unmapped_rows], axis=0)

In [32]:
df_concat.shape

(5984, 10)

In [33]:
df_concat.isna().sum()

head_id      0
head_name    0
head_type    0
head_ref     0
tail_id      0
tail_name    0
tail_type    0
tail_ref     0
predicate    0
status       0
dtype: int64

In [34]:
df_concat.head(1)

,head_id,head_name,head_type,head_ref,tail_id,tail_name,tail_type,tail_ref,predicate,status
10,DB08604,Triclosan,drug,DrugBank,673.0,Malaria,disease,Orphanet,ddii,Approved


Separating into Positive and Negative Pairs

In [35]:
keywords = ['Discontinued', 'Withdrawn', 'Terminated']

In [36]:
discontinued_df = df_concat[df_concat['status'].str.contains('|'.join(keywords), case=False, na=False)]

In [37]:
discontinued_df.shape

(451, 10)

In [38]:
active_df = df_concat[~df_concat['status'].str.contains('|'.join(keywords), case=False, na=False)]

In [39]:
active_df.shape

(5533, 10)

In [40]:
discontinued_df = discontinued_df.iloc[:, :-1]
active_df = active_df.iloc[:, :-1]

In [41]:
discontinued_df['predicate'] = discontinued_df['predicate'].replace('ddii', 'nddii')

In [42]:
discontinued_df.head(1)

,head_id,head_name,head_type,head_ref,tail_id,tail_name,tail_type,tail_ref,predicate
111,DB09073,Palbociclib,drug,DrugBank,3140.0,Schizophrenia,disease,Orphanet,nddii


In [43]:
active_df.head(1)

,head_id,head_name,head_type,head_ref,tail_id,tail_name,tail_type,tail_ref,predicate
10,DB08604,Triclosan,drug,DrugBank,673.0,Malaria,disease,Orphanet,ddii


In [ ]:
discontinued_df.to_csv(r'...path_to\RDsqr-KG\KG_datasets\nddii.csv', index=False)
active_df.to_csv(r'...path_to\RDsqr-KG\KG_datasets\ddii.csv', index=False)